[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/finance-data-analysis/blob/main/3.3%20%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B8%88%EC%9C%B5%20%EA%B0%9C%EB%B3%84%EC%A2%85%EB%AA%A9%20%EC%88%98%EC%A7%91-input.ipynb)


## 네이버 금융 개별종목 수집
* FinanceDataReader를 통해 수집했던 데이터를 네이버 증권 웹 페이지를 통해 직접 수집합니다.


### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 2020년 주요 상장종목
    * 빅히트 : https://finance.naver.com/item/main.nhn?code=352820
    * 카카오게임즈 : https://finance.naver.com/item/main.nhn?code=293490
    * SK바이오팜 : https://finance.naver.com/item/main.nhn?code=326030

메모
* https://finance.naver.com/item/sise_day.nhn?code=005930&page=6

*삼성전자를 원하면 005930, 다른 거를 원하면 그 회사 코드를 쓰면 됨 

## 라이브러리 로드

In [4]:
# 라이브러리 로드
import pandas as pd
import numpy as np

## 수집할 URL 정하기

In [15]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "005930"
# item_name = "하이브"

item_code = "326030"
item_name = "SK바이오팜"

# 종목 URL 만들기
url = 'https://finance.naver.com/item/sise_day.nhn?code=005930&page=3'

## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [43]:
import requests
requests.__version__

'2.24.0'

In [47]:
# 왜 url을 안보내 주지?
#response = requests.get(url)
#response.text

'\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 :: 세상의 모든 지식, 네이버</title>\n\n<style type="text/css">\n.error_content * {margin:0;padding:0;}\n.error_content img{border:none;}\n.error_content em {font-style:normal;}\n.error_content {width:410px; margin:80px auto 0; padding:57px 0 0 0; font-size:12px; font-family:"나눔고딕", "NanumGothic", "돋움", Dotum, AppleGothic, Sans-serif; text-align:left; line-height:14px; background:url(https://ssl.pstatic.net/static/common/error/090610/bg_thumb.gif) no-repeat center top; white-space:nowrap;}\n.error_content p{margin:0;}\n.error_content .error_desc {margin-bottom:21px; overflow:hidden; text-align:center;}\n.error_content .error_desc2 {margin-bottom:11px; padding-bottom:7px; color:#888; line-height:18px; border-bottom:1px solid #eee;}\n.error_content .error_desc3 {clear:both; color:#888;}\n.error_con

In [50]:
# 그렇다면 user-agent를 가져오자!
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36"}
print(type(headers))
headers

# dict 형태로 나오는지 꼭 확인하기

<class 'dict'>


{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'}

In [49]:
# headers 를 추가하자
response = requests.get(url, headers = headers)
response.text

'\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 금융</title>\n\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/newstock.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/common.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/layout.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/main.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/newstock2.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/newstock3.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/world.css">\n</head>\n<body>\n<script

## BeautifulSoup 을 통한 table 태그 찾기

* [Beautiful Soup Documentation — Beautiful Soup 4.9.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [51]:
from bs4 import BeautifulSoup as bs
# import bs4
# bs4.__version__

html = bs(response.text, 'lxml')

In [55]:
tables = html.select("table")
len(tables)

2

In [24]:
import tqdm
tqdm.__version__

'4.50.2'

## pandas 코드 한 줄로 데이터 수집하기

In [17]:
print(url)

https://finance.naver.com/item/sise_day.nhn?code=005930&page=3


In [14]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.
# table

pd.read_html(url, encoding = 'cp949')

# 'euc-kr'와 'cp949' 의 차이: euc-kr은 2300자까지만 지원, but 한글은 17072자 
# https://www.hani.co.kr/arti/society/society_general/864914.html

[                    0                                          1  \
 0  전일  78,800  78,800  고가  79,40079,400  (상한가  102,000102,000  )   
 1    시가  79,20079,200          저가  78,60078,600  (하한가  55,200  )   
 
                             2  
 0   거래량  8,584,497  8,584,497  
 1  거래대금  677,556  677,556  백만  ,
            0            1       2           3
 0        현재가        78700    매도호가       78800
 1       전일대비      하락  100    매수호가       78700
 2     등락률(%)       -0.13%     전일가       78800
 3        거래량      8584497      시가       79200
 4   거래대금(백만)       677556      고가       79400
 5        액면가         100원      저가       78600
 6        NaN          NaN     NaN         NaN
 7        상한가       102000    전일상한      103000
 8        하한가        55200    전일하한       55600
 9        PER        18.90     EPS        4165
 10    52주 최고        96800  52주 최저       54000
 11      시가총액  4,698,219억원   상장주식수  5969782550
 12     외국인현재  3,181,799천주     자본금   778,046백만,
         매도잔량     매도호가  Unnamed:

In [56]:
table = pd.read_html(str(tables))
table

[            날짜       종가     전일비       시가       고가       저가         거래량
 0          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 1   2021.06.29  81000.0   900.0  81900.0  82100.0  80800.0  15744317.0
 2   2021.06.28  81900.0   300.0  81700.0  82000.0  81600.0  11578529.0
 3   2021.06.25  81600.0   400.0  81500.0  81900.0  81200.0  13481405.0
 4   2021.06.24  81200.0  1100.0  80400.0  81400.0  80100.0  18771080.0
 5   2021.06.23  80100.0   100.0  80500.0  80600.0  79900.0  13856548.0
 6          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 7          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 8          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 9   2021.06.22  80000.0   100.0  80200.0  80300.0  79900.0  11773365.0
 10  2021.06.21  79900.0   600.0  79700.0  80000.0  79600.0  16063340.0
 11  2021.06.18  80500.0   400.0  81100.0  81100.0  80500.0  14916721.0
 12  2021.06.17  80900.0   900.0  81100.0  81300.0  80700.0  140

In [62]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[0]



,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021.06.29,81000.0,900.0,81900.0,82100.0,80800.0,15744317.0
2,2021.06.28,81900.0,300.0,81700.0,82000.0,81600.0,11578529.0
3,2021.06.25,81600.0,400.0,81500.0,81900.0,81200.0,13481405.0
4,2021.06.24,81200.0,1100.0,80400.0,81400.0,80100.0,18771080.0
5,2021.06.23,80100.0,100.0,80500.0,80600.0,79900.0,13856548.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021.06.22,80000.0,100.0,80200.0,80300.0,79900.0,11773365.0


In [63]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
temp = table[0].dropna()
temp

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.06.29,81000.0,900.0,81900.0,82100.0,80800.0,15744317.0
2,2021.06.28,81900.0,300.0,81700.0,82000.0,81600.0,11578529.0
3,2021.06.25,81600.0,400.0,81500.0,81900.0,81200.0,13481405.0
4,2021.06.24,81200.0,1100.0,80400.0,81400.0,80100.0,18771080.0
5,2021.06.23,80100.0,100.0,80500.0,80600.0,79900.0,13856548.0
9,2021.06.22,80000.0,100.0,80200.0,80300.0,79900.0,11773365.0
10,2021.06.21,79900.0,600.0,79700.0,80000.0,79600.0,16063340.0
11,2021.06.18,80500.0,400.0,81100.0,81100.0,80500.0,14916721.0
12,2021.06.17,80900.0,900.0,81100.0,81300.0,80700.0,14007385.0
13,2021.06.16,81800.0,900.0,81500.0,81900.0,81100.0,14999855.0


## 페이지별 데이터 수집 함수 만들기

In [64]:
print(url)

https://finance.naver.com/item/sise_day.nhn?code=005930&page=3


In [65]:
headers

{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'}

In [71]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

def get_day_list(item_code, page_no):
# """
# 일자별 시세를 페이지별로 수집
# """ 
    url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'}
    response = requests.get(url, headers = headers)
    html = bs(response.text, 'lxml')
    tables = html.select("table")
    table =  pd.read_html(str(tables))
    df_page = table[0].dropna()
    return pf_page

In [72]:
# 함수가 잘 만들어졌는지 확인하기
page_no = 22
item_code = '068270'
get_day_list(item_code, page_no)

NameError: name 'pf_page' is not defined

## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [34]:
import time
# web page 시작번호
page_no = 1
# 데이터를 저장할 빈 변수 선언
item_list = []


## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

[1, 2, 3, 4]